In [35]:
import os
import torch
from pytube import YouTube
from transformers import pipeline


In [45]:
import re

def get_video_id(video_url):
    """
    Extracts the YouTube Video ID from different types of YouTube URLs.
    """
    pattern = r"(?:v=|\/(?:embed|shorts|v)\/|youtu\.be\/)([^#\&\?]+)"
    match = re.search(pattern, video_url)
    return match.group(1) if match else None

# 🔹 Test the function
video_url = input("Enter YouTube Video URL: ")
video_id = get_video_id(video_url)

if video_id:
    print("✅ Extracted Video ID:", video_id)
else:
    print("❌ Invalid YouTube URL")


✅ Extracted Video ID: aDG1T0kJnd4


In [46]:
from youtube_transcript_api import YouTubeTranscriptApi

def get_transcript(video_id):
    """
    Fetches the transcript of a given YouTube video ID.
    """
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        full_text = "\n".join([entry["text"] for entry in transcript])  # Combine all transcript lines
        return full_text
    except Exception as e:
        return f"❌ Error fetching transcript: {str(e)}"

# 🔹 Fetch transcript for the extracted Video ID
if video_id:
    transcript_text = get_transcript(video_id)
    print("\n📜 Full Transcript:\n")
    print(transcript_text[:10000])  # Display only the first 1000 characters (adjust if needed)



📜 Full Transcript:

Thank you.
Thank you.
So, my father used to always tell me something
which I want to share with you, that why do
you want to fit inside a glass slipper?
You know, like we were told, like Cinderella
did, why do you want to fit inside a glass
slipper when you can shatter the glass ceiling?
I want to tell you a little secret.
I'm not very fond of this phrase, breaking
the glass ceiling.
Why does it annoy me?
Because it takes the context of everything
that I have done.
All my achievements, all my hard work and
puts it into a box as if my ambition was that
I want to find a glass ceiling and break it.
Not at all.
To be really honest, I was never on a mission
to break, to shatter anything.
All I wanted was to chase my dreams, my ambitions.
I wanted to evolve.
I wanted to become the best version of me,
that I could be.
And of course, in Priyanka Chopra style, along
the way, I wanted to smash and break every
obstacle that came my way, which I did.
But that does not mean tha

In [47]:
def ask_for_summary(transcript):
    """
    Ask the user whether they want a summary of the transcript.
    """
    print("\n⚡ Transcript fetched successfully!")
    user_choice = input("🔹 Do you want a summary of this transcript? (yes/no): ").strip().lower()

    if user_choice == "yes":
        return True
    else:
        print("✅ Transcript displayed. No summary generated.")
        return False

# 🔹 Ask the user for summary
wants_summary = ask_for_summary(transcript_text)



⚡ Transcript fetched successfully!


In [49]:
# Function to generate summary
def generate_summary(transcript):
    """
    Generates a summary using the Facebook BART model.
    """
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    if len(transcript.split()) < 50:  # Ensure we have enough words
        return "Transcript is too short for summarization."

    # Limiting input to BART model's max length (1024 tokens)
    max_input_length = 1024
    transcript = transcript[:max_input_length]  # Truncate if too long

    try:
        summary = summarizer(transcript, max_length=300, min_length=100, do_sample=False)[0]['summary_text']
        return summary
    except Exception as e:
        return f"Error generating summary: {str(e)}"
